In [1]:
import os
import json
import nibabel as nib
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import numpy as np
from LazyLuna.utils import *
from LazyLuna.Mini_LL import *
from LazyLuna.Views import *

import pydicom
from pydicom.dataset import Dataset
import pydicom._storage_sopclass_uids

In [9]:
# from nifti path
basepath = '/Users/dietrichhadler/Desktop/Thomas/LL_Test'
base_nii = os.path.join(basepath, 'emidec-dataset-1.0.1')

# to dicoms and annotation paths
imgs_path = os.path.join(basepath, 'Imgs')
gold_path = os.path.join(basepath, 'Gold')
ai_path   = os.path.join(basepath, 'Emidec_AI')
# to cases
cases_path = os.path.join(basepath, 'Cases')
for p in [imgs_path, gold_path, cases_path]: os.mkdir(p) if not os.path.exists(p) else pass

# load dictionary of case name --> studyuid and sopinstanceuids of slices
with open(os.path.join(basepath, 'case_slice_to_sops.json'), 'r') as fp:
    sort_dict = json.load(fp)

In [3]:
###################
# Transform Annos #
###################
def emidec_to_anno_dict(mask, img_size, pixel_size):
    # 1: endo # 2: healthy myo # 3: scar # 4: no reflow
    endo_cont     = to_polygon((mask==1).astype(np.int16))
    myo_cont      = to_polygon((mask>=2).astype(np.int16))
    scar_cont     = to_polygon((mask==3).astype(np.int16))
    noreflow_cont = to_polygon((mask==4).astype(np.int16))
    anno_dict = dict()
    if not endo_cont.is_empty:     anno_dict['lv_endo']  = {'cont': endo_cont}
    if not myo_cont.is_empty :     anno_dict['lv_myo']   = {'cont': myo_cont}
    if not scar_cont.is_empty:     anno_dict['scar']  = {'cont': scar_cont}
    if not noreflow_cont.is_empty: anno_dict['noreflow'] = {'cont': noreflow_cont}
    anno_dict['info'] = {'imageSize': img_size, 'pixelSize': pixel_size}
    return anno_dict

def emidec_transform_to_ll_annos(nii_annos, bpath, studyiuid, sops):
    contdir = os.path.join(bpath, studyiuid) 
    if not os.path.exists(contdir): os.mkdir(contdir)
    h , w, nr_slices = nii_annos.shape
    ph, pw, slice_th = nii_annos.header['pixdim'][1:4]
    mask_data = nii_annos.get_fdata().astype(np.int32)
    ll_annos = []
    for d in range(nr_slices):
        ll_annos.append(emidec_to_anno_dict(mask_data[:,:,d], [h,w], [ph,pw]))
        with open(os.path.join(contdir, sops[d]+'.pickle'), 'wb') as f:
            pickle.dump(ll_annos[-1], f, pickle.HIGHEST_PROTOCOL)
    return ll_annos        

In [4]:
def nifti_to_dcm(nii_imgs, bpath, casename, studyuid, sops):
    if not os.path.exists(os.path.join(bpath, casename)):
        os.mkdir(os.path.join(bpath, casename))
    h, w, nr_slices  = nii_imgs.header['dim'][1:4]
    ph, pw, pdepth = nii_imgs.header['pixdim'][1:4]
    imgs = nii_imgs.get_fdata()
    for d in range(nr_slices):
        img = imgs[:,:,d]
        img = img.astype(np.uint16)
        meta = pydicom.Dataset()
        meta.MediaStorageSOPClassUID = pydicom._storage_sopclass_uids.MRImageStorage
        meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
        meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
        ds = Dataset()
        ds.file_meta = meta
        ds.SOPInstanceUID = sops[d]
        ds.is_little_endian = True
        ds.is_implicit_VR = False
        ds.SOPClassUID = pydicom._storage_sopclass_uids.MRImageStorage
        ds.PatientName = 'PatientX'
        ds.PatientID = "123456"
        ds.Modality = "MR"
        ds.SeriesInstanceUID = pydicom.uid.generate_uid()
        ds.StudyInstanceUID  = studyuid
        ds.FrameOfReferenceUID = pydicom.uid.generate_uid()
        ds.SeriesDescription = "sax lge"
        ds.SliceLocation  = d*pdepth
        ds.SliceThickness = str(pdepth)
        ds.SpacingBetweenSlices = str(pdepth)
        ds.PixelSpacing = str(ph)+'\\'+str(pw)
        ds.SeriesNumber = 0
        ds.BitsStored = 16
        ds.BitsAllocated = 16
        ds.SamplesPerPixel = 1
        ds.HighBit = 15
        ds.ImagesInAcquisition = "1"
        ds.Rows    = h
        ds.Columns = w
        ds.InstanceNumber = 0
        ds.ImagePositionPatient = r"0\0\1"
        ds.ImageOrientationPatient = r"1\0\0\0\-1\0"
        ds.ImageType = r"ORIGINAL\PRIMARY\AXIAL"
        ds.RescaleIntercept = "0"
        ds.RescaleSlope     = "1"
        ds.PhotometricInterpretation = "MONOCHROME2"
        ds.PixelRepresentation = 1
        pydicom.dataset.validate_file_meta(ds.file_meta, enforce_standard=True)
        ds.PixelData = img.tobytes()
        ds.private_block(0x000b, 'Lazy Luna: SAX LGE', create=True)
        filename = os.path.join(bpath, casename, str(d)+'.dcm')
        ds.save_as(filename=filename, write_like_original=False)

In [5]:
folders = [f for f in os.listdir(base_nii) if os.path.isdir(os.path.join(base_nii, f))]
for f in folders:
    img_path  = os.path.join(base_nii, f, 'Images')
    anno_path = os.path.join(base_nii, f, 'Contours')
    nii_imgs  = nib.load(os.path.join(img_path,  f+'.nii.gz'))
    nii_annos = nib.load(os.path.join(anno_path, f+'.nii.gz'))
    studyiuid, sops = sort_dict[f]['study_uid'], sort_dict[f]['sopinstanceuids']
    nifti_to_dcm(nii_imgs, imgs_path, f, studyiuid, sops)
    annos = emidec_transform_to_ll_annos(nii_annos, gold_path, studyiuid, sops)

In [8]:
# transform cases to LL # can take a minute
paths = get_imgs_and_annotation_paths(imgs_path, gold_path)
view = SAX_LGE_View()
for imgp, annop in paths:
    print('.', end='')
    case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(gold_path))
    view.initialize_case(case, cvi_preprocess=False, debug=False)
    case.store(cases_path)

....................................................................................................

In [7]:
# transform AI cases to LL # can take a minute
paths = get_imgs_and_annotation_paths(imgs_path, ai_path)
view = SAX_LGE_View()
for imgp, annop in paths:
    print('.', end='')
    case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(ai_path))
    view.initialize_case(case, cvi_preprocess=False, debug=False)
    case.store(cases_path)

....................................................................................................

In [ ]:
cases = [pickle.load(open(os.path.join(cases_path, p), 'rb')) for p in os.listdir(cases_path)]
v     = SAX_LGE_View()
cases = [v.customize_case(c) for c in cases]
for i in range(50, 100):
    case  = cases[i] # 50th is the first patient -with- scars
    cat   = case.categories[0]
    for d in range(cat.nr_slices):
        img, anno = cat.get_img(d,0), cat.get_anno(d,0)
        fig, ax = plt.subplots(1,1,figsize=(7,7))
        b   = 35
        bb1 = anno.get_contour('lv_myo').envelope
        bb2 = anno.get_contour('lv_myo').envelope
        bb  = bb1 if hasattr(bb1, 'exterior') else bb2
        if hasattr(bb, 'exterior'):
            x, y = np.array(bb.exterior.xy)
            lx, ly, ux, uy = x.min()-b-10, y.min()-b, x.max()+b, y.max()+b
            ax.set_xlim([lx, ux]); ax.set_ylim([ly, uy]); ax.invert_yaxis()
        ax.axis('off')
        ax.imshow(img, cmap='gray')
        #anno.plot_all_contour_outlines(ax)
        anno.plot_contour_outlines(ax, 'lv_myo')
        anno.plot_contour_face    (ax, 'lv_myo'  , 'w')
        anno.plot_contour_face    (ax, 'scar'    , 'r')
        anno.plot_contour_face    (ax, 'noreflow', 'b')
        plt.show()